In [1]:
import time
from absl import logging
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import tqdm
import os

In [2]:
%run 'yolov3_tf2.ipynb'

In [3]:
%run 'utils.ipynb'

In [4]:
classFile = './data/classes.txt'
weights = './checkpoints/yolov3_train_50.tf'
size = 416
#image = './data/20181107133258_camera_frontcenter_000000250.png'
tfrecord =  None#'./data/audi_test.tfrecord'
#output = './output.jpg'
num_classes = 14
class_names = [c.strip() for c in open(classFile).readlines()]

def detect(image):
    
    yolo = YoloV3(classes=num_classes)

    yolo.load_weights(weights).expect_partial()
    logging.info('weights loaded')

    #if tfrecord:
    #    dataset = load_tfrecord_dataset(
    #        tfrecord, classFile, size)
    #    dataset = dataset.shuffle(512)
    #    img_raw, _label = next(iter(dataset.take(1)))
    #else:
    img_raw = tf.image.decode_image(open(image, 'rb').read(), channels=3)

    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, size)

    t1 = time.time()
    boxes, scores, classes, nums = yolo(img)
    t2 = time.time()
    logging.info('time: {}'.format(t2 - t1))

    #logging.info('detections:')
    #for i in range(nums[0]):
        #logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                           #np.array(scores[0][i]),
                                           #np.array(boxes[0][i])))

    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
        
    #img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
    wh = np.flip(img.shape[0:2])
    #cv2.imwrite(output, img)
    #fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(15,15))
    #plt.imshow(img)
    #logging.info('output saved to: {}'.format(output))
    return boxes, scores, classes, nums, wh

In [5]:
test_dataset = pd.read_csv('test.csv', index_col=0)
test_dataset.head(3)

,id,imgFile,jsonFile
1192,29293,/Data/Audi_dataset/20180810_142822/camera/cam_...,/Data/Audi_dataset/20180810_142822/label3D/cam...
5374,51273,/Data/Audi_dataset/20181008_095521/camera/cam_...,/Data/Audi_dataset/20181008_095521/label3D/cam...
4723,38484,/Data/Audi_dataset/20180925_135056/camera/cam_...,/Data/Audi_dataset/20180925_135056/label3D/cam...


In [6]:
cur_dir = os.getcwd()
idx = 0
df = pd.DataFrame(columns=['id','imgFile', 'x1', 'x2', 'y1', 'y2','class','score'])
d = pd.DataFrame()
for key, row in tqdm.tqdm(test_dataset[0:50].iterrows()):
    outputs = detect(cur_dir+row['imgFile'])
    boxes, objectness, classes, nums, wh = outputs
    boxes, objectness, classes, nums = boxes[0], objectness[0], classes[0], nums[0]

    for i in range(nums):
        x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
        x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
        df.loc[idx] = pd.Series({'id':row['id'],'imgFile': row['imgFile'], 'x1': x1y1[0], 
                                 'x2': x2y2[0], 'y1':x1y1[1] , 'y2': x2y2[1],
                                 'class':class_names[int(classes[i])],
                                 'score':round(float(objectness[i]),3) })
        idx += 1 

50it [08:41, 17.95s/it]


In [20]:
df.shape

(117, 8)

In [10]:
df.to_csv('data/test_observation.csv')
arr = df.id.unique()
audiDf = pd.read_csv('data/audi_dataset.csv', index_col=0)

In [19]:
a = audiDf[audiDf.id.isin(arr)]
a.shape

(169, 11)

In [23]:
df.sort_values(['id','class','x1'],ascending= True)

,id,imgFile,x1,x2,y1,y2,class,score
97,1178,/Data/Audi_dataset/20181107_133258/camera/cam_...,873,1115,714,836,Car,0.577
99,1311,/Data/Audi_dataset/20181107_132730/camera/cam_...,225,419,707,816,Car,0.816
98,1311,/Data/Audi_dataset/20181107_132730/camera/cam_...,970,1068,716,778,Car,0.993
6,2282,/Data/Audi_dataset/20181107_132300/camera/cam_...,782,1314,753,1089,Car,0.938
8,3670,/Data/Audi_dataset/20181107_132300/camera/cam_...,935,1051,694,784,Car,0.996
9,3670,/Data/Audi_dataset/20181107_132300/camera/cam_...,1086,1171,684,752,Car,0.968
108,3684,/Data/Audi_dataset/20181107_132300/camera/cam_...,921,1048,703,794,Car,0.999
109,3684,/Data/Audi_dataset/20181107_132300/camera/cam_...,1075,1185,689,767,Car,0.998
73,4502,/Data/Audi_dataset/20181107_132300/camera/cam_...,881,1094,695,844,Car,0.951
70,4502,/Data/Audi_dataset/20181107_132300/camera/cam_...,1117,1252,702,796,Car,0.980


In [29]:
a.x1 = a.x1.astype(int)
a.x2 = a.x2.astype(int)
a.y1 = a.y1.astype(int)
a.y2 = a.y2.astype(int)
a.sort_values(['id','class','x1'],ascending= True)[['id','imageFile','x1','y1','x2','y2','class']]

,id,imageFile,x1,y1,x2,y2,class
25473,1178,/data/Audi_dataset/20181107_133258/camera/cam_...,903,709,1615,923,Car
38178,1311,/data/Audi_dataset/20181107_132730/camera/cam_...,228,710,441,808,Car
38181,1311,/data/Audi_dataset/20181107_132730/camera/cam_...,977,715,1051,779,Car
38180,1311,/data/Audi_dataset/20181107_132730/camera/cam_...,1631,587,2102,1174,Cyclist
38179,1311,/data/Audi_dataset/20181107_132730/camera/cam_...,865,711,939,773,VanSUV
34213,2282,/data/Audi_dataset/20181107_132300/camera/cam_...,309,661,486,729,Car
34211,2282,/data/Audi_dataset/20181107_132300/camera/cam_...,823,733,1278,1168,Car
34212,2282,/data/Audi_dataset/20181107_132300/camera/cam_...,1518,739,2418,1276,Car
34403,3670,/data/Audi_dataset/20181107_132300/camera/cam_...,935,695,1050,792,Car
34404,3670,/data/Audi_dataset/20181107_132300/camera/cam_...,1083,691,1161,751,Car
